# OpenFermion Fermionic Quantum Emulator 
## Phase I Design Proposal

This document outlines the functionality of the API for the OpenFermion Fermionic Quantum Emulator as defined in the design proposal.

 The code is covered and tested with pytest-cov at 99%. Toggle the print option to see output.

In [2]:
fqeprint = True

### 2.A.2 API for data conversion between OpenFermion's backend and OpenFermion-FQE
The API supports data conversion between native Cirq wavefunctions and the FQE wavefunction for arbitrary particle number and spin configuration.  A linequbit wavefunction result is a normalized numpy array.

In [3]:
import fqe
import numpy
import cirq

nqubit = 4

cirq_wfn = fqe.util.rand_wfn(2**nqubit).astype(numpy.complex64)
wfn_from_cirq = fqe.from_cirq(cirq_wfn, 0.0001)

if fqeprint:
    print('Inital wavefunction is \n {} \n with norm {}'.format(cirq_wfn, numpy.vdot(cirq_wfn, cirq_wfn)))
    wfn_from_cirq.print_wfn()

Inital wavefunction is 
 [-0.00176808+0.2826661j  -0.14123131+0.008422j   -0.0953294 +0.21192858j
  0.06359234+0.11356124j  0.23168676-0.05730962j -0.2716447 -0.13480759j
 -0.07564119+0.07610556j -0.13439253+0.5034181j  -0.18308303-0.15851946j
 -0.20814021-0.07259662j  0.02144353+0.04388449j  0.04466553+0.09139249j
 -0.04765405+0.22987317j -0.10665143+0.22184473j -0.07224102-0.22913909j
  0.00032615+0.32219037j] 
 with norm (1+0j)

Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (-0.0017680843593552709+0.28266608715057373j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (0.2316867560148239-0.05730962008237839j)
 a'00'b'10' : (-0.14123131334781647+0.008421998471021652j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (-0.18308302760124207-0.1585194617509842j)
 a'10'b'00' : (-0.0953293964266777+0.2119285762310028j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.2716447114944458-0.13480758666992188j)

Configuration nelectrons: 2 m

### 2.A.3 Initialization of wave functions
FQE wave functions can be initialized by calling the constructor directly. fqe.get_wavefunction and get_wavefunction_mutltiple are utility wrappers that provide wrappers for common initialization motifs. e.g. get_spin_nonconserving_wavefunction will build a wavefunction that permits spin breaking Operators to be applied while get_wavefunction_multiple will return a list of initialize wavefunctions

In [4]:
wfFqe = fqe.Wavefunction([[4, -2, 10]])
wfFqe = fqe.get_wavefunction(4, -2, 10)

In [5]:
n = 3
m = -2
wfFQE = fqe.Wavefunction([[n, m, 10], [n+1, m+2, 10]])
wfFQE = fqe.get_wavefunction_multiple([[4, 0, 10], [5, -5, 10]])

In [6]:
wfFQE = fqe.get_spin_nonconserving_wavefunction(2)
wfFQE.set_wfn(strategy='random')
if fqeprint:
    wfFQE.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (0.1321912258863449+0.2478310614824295j)
 a'0001'b'0010' : (-0.3915639817714691+0.10769174247980118j)
 a'0001'b'0100' : (-0.16823995113372803-0.19465914368629456j)
 a'0001'b'1000' : (-0.32013314962387085+0.14106538891792297j)
 a'0010'b'0001' : (-0.07473369687795639-0.22307895123958588j)
 a'0010'b'0010' : (-0.34121984243392944+0.17437410354614258j)
 a'0010'b'0100' : (0.1934564709663391+0.04763473942875862j)
 a'0010'b'1000' : (-0.29369011521339417-0.1090521588921547j)
 a'0100'b'0001' : (-0.040231697261333466+0.04645397514104843j)
 a'0100'b'0010' : (0.24862094223499298+0.0076537709683179855j)
 a'0100'b'0100' : (0.00208487780764699-0.0010802372125908732j)
 a'0100'b'1000' : (0.11801540106534958-0.01888784021139145j)
 a'1000'b'0001' : (-0.04116491228342056-0.10742396116256714j)
 a'1000'b'0010' : (0.037345319986343384+0.02972235158085823j)
 a'1000'b'0100' : (0.3301687240600586-0.10125236213207245j)
 a'1000'b'1000' : (0.039315067

### 2.A.4 Element access

Element access is modified from the original propsal to add more convient getter and setter functionalities.  The orginally proposed methods are availible with an addition function to add a value to an existing element.

In [7]:
wfFQE.set_ele(1, 8, .14159265359j)
wfFQE.add_ele(1, 8, 3.j)
imaginary_pi = wfFQE.get_ele(1, 8)
if fqeprint:
    print(-imaginary_pi*1.j)

(3.1415927410125732-0j)


The new components not originally in the proposal allow access of entire vectors for getting and setting methods.

In [8]:
interesting_states = wfn_from_cirq.get_coeff((2, 0), vec=[0,2])
if fqeprint:
    print(interesting_states)

[[-0.04765405+0.22987317j  0.        +0.j        ]
 [-0.20814021-0.07259662j  0.        +0.j        ]
 [ 0.07564119-0.07610556j  0.        +0.j        ]
 [ 0.06359234+0.11356124j  0.        +0.j        ]]


In [9]:
interesting_states[:, 1] += 1. + .1j
interesting_states *= .7 - .3j
if fqeprint:
    print(interesting_states)

[[ 0.03560412+0.17520744j  0.73      -0.23000002j]
 [-0.16747713+0.01162443j  0.73      -0.23000002j]
 [ 0.03011717-0.07596625j  0.73      -0.23000002j]
 [ 0.07858302+0.06041516j  0.73      -0.23000002j]]


In [10]:
data = {}
data[(2, 0)] = interesting_states
wfn_from_cirq.set_wfn(vrange=[0, 2], strategy='from_data', raw_data=data)
if fqeprint:
    wfn_from_cirq.print_wfn()


Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (-0.0017680843593552709+0.28266608715057373j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (0.2316867560148239-0.05730962008237839j)
 a'00'b'10' : (-0.14123131334781647+0.008421998471021652j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (-0.18308302760124207-0.1585194617509842j)
 a'10'b'00' : (-0.0953293964266777+0.2119285762310028j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.2716447114944458-0.13480758666992188j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.035604123026132584+0.17520743608474731j)
 a'01'b'10' : (-0.16747713088989258+0.011624429374933243j)
 a'10'b'01' : (0.03011716529726982-0.0759662464261055j)
 a'10'b'10' : (0.0785830169916153+0.06041516363620758j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (0.021443530917167664+0.04388448968529701j)

Configuration nelectrons: 3 m_s: -1
Vector : 0
 a'01'b'11' : (-0.1066514328122139+0

### 2.A.5 Printing/Saving

Printing is currently availible as alpha beta strings followed by the coefficient as well as orbital occupation representation.

In [11]:
print('String forrmatting')
wfn_from_cirq.print_wfn(fmt='str')
print('\n Occupation forrmatting')
wfn_from_cirq.print_wfn(fmt='occ')

String forrmatting

Configuration nelectrons: 0 m_s: 0
Vector : 0
 a'00'b'00' : (-0.0017680843593552709+0.28266608715057373j)

Configuration nelectrons: 1 m_s: -1
Vector : 0
 a'00'b'01' : (0.2316867560148239-0.05730962008237839j)
 a'00'b'10' : (-0.14123131334781647+0.008421998471021652j)

Configuration nelectrons: 1 m_s: 1
Vector : 0
 a'01'b'00' : (-0.18308302760124207-0.1585194617509842j)
 a'10'b'00' : (-0.0953293964266777+0.2119285762310028j)

Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.2716447114944458-0.13480758666992188j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.035604123026132584+0.17520743608474731j)
 a'01'b'10' : (-0.16747713088989258+0.011624429374933243j)
 a'10'b'01' : (0.03011716529726982-0.0759662464261055j)
 a'10'b'10' : (0.0785830169916153+0.06041516363620758j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (0.021443530917167664+0.04388448968529701j)

Configuration nelectrons: 3 m_s: -1
Vector : 0
 a'01'b'11' : (-0

### 2.A.6 Fermionic algebra operations and their unitaries on the state

FermionOperators can be directly passed in to create a newwavefunction based on application of the operators.

In [12]:
from openfermion import FermionOperator
ops = FermionOperator('2^ 0', 1.2)
new_wfn = wfn_from_cirq.apply(ops)

In [13]:
new_wfn = fqe.apply(FermionOperator('0^ 0', 1.0), wfFQE)
if fqeprint:
    wfFQE.print_wfn()
    new_wfn.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (0.1321912258863449+0.2478310614824295j)
 a'0001'b'0010' : (-0.3915639817714691+0.10769174247980118j)
 a'0001'b'0100' : (-0.16823995113372803-0.19465914368629456j)
 a'0001'b'1000' : 3.1415927410125732j
 a'0010'b'0001' : (-0.07473369687795639-0.22307895123958588j)
 a'0010'b'0010' : (-0.34121984243392944+0.17437410354614258j)
 a'0010'b'0100' : (0.1934564709663391+0.04763473942875862j)
 a'0010'b'1000' : (-0.29369011521339417-0.1090521588921547j)
 a'0100'b'0001' : (-0.040231697261333466+0.04645397514104843j)
 a'0100'b'0010' : (0.24862094223499298+0.0076537709683179855j)
 a'0100'b'0100' : (0.00208487780764699-0.0010802372125908732j)
 a'0100'b'1000' : (0.11801540106534958-0.01888784021139145j)
 a'1000'b'0001' : (-0.04116491228342056-0.10742396116256714j)
 a'1000'b'0010' : (0.037345319986343384+0.02972235158085823j)
 a'1000'b'0100' : (0.3301687240600586-0.10125236213207245j)
 a'1000'b'1000' : (0.03931506723165512-0.1063833534717

The fqe has vdot functionality to get an inner product and provides a way to do completely general expectation values.

In [14]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, wfFQE.apply(ops))
if fqeprint:
    print(opexpec)

(0.1700136959552765+0j)


In [15]:
ops = FermionOperator('2^ 2', 0.5)
opexpec = fqe.vdot(wfFQE, fqe.apply(ops, wfFQE))
if fqeprint:
    print(opexpec)

(0.1700136959552765+0j)


We can also do general unitary application of operators.  One slight modification to the proposal is that the operator passed in should Hermetian and not anti-Hermitian as stated before.

In [16]:
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE.print_wfn()
wfFQE2 = wfFQE.apply_generated_unitary(ops, algo='taylor')
wfFQE2.print_wfn()
ops = FermionOperator('2^ 0', 0.2 - .3j) + FermionOperator('0^ 2', 0.2 + .3j)
wfFQE2 = fqe.apply_generated_unitary(ops, wfFQE, algo='taylor')
wfFQE2.print_wfn()


Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'0001'b'0001' : (0.1321912258863449+0.2478310614824295j)
 a'0001'b'0010' : (-0.3915639817714691+0.10769174247980118j)
 a'0001'b'0100' : (-0.16823995113372803-0.19465914368629456j)
 a'0001'b'1000' : 3.1415927410125732j
 a'0010'b'0001' : (-0.07473369687795639-0.22307895123958588j)
 a'0010'b'0010' : (-0.34121984243392944+0.17437410354614258j)
 a'0010'b'0100' : (0.1934564709663391+0.04763473942875862j)
 a'0010'b'1000' : (-0.29369011521339417-0.1090521588921547j)
 a'0100'b'0001' : (-0.040231697261333466+0.04645397514104843j)
 a'0100'b'0010' : (0.24862094223499298+0.0076537709683179855j)
 a'0100'b'0100' : (0.00208487780764699-0.0010802372125908732j)
 a'0100'b'1000' : (0.11801540106534958-0.01888784021139145j)
 a'1000'b'0001' : (-0.04116491228342056-0.10742396116256714j)
 a'1000'b'0010' : (0.037345319986343384+0.02972235158085823j)
 a'1000'b'0100' : (0.3301687240600586-0.10125236213207245j)
 a'1000'b'1000' : (0.03931506723165512-0.1063833534717

### 3.B.1 Hamiltonian data structure

Hamiltonians can be incorporated from OpenFermion objects or initialized from numpy arrays.  Notice a slight change in the arguments to get_two_body to include a constant potential term and additionally a chemical potential to be consistent with OpenFermion.

In [17]:
from openfermion import get_quadratic_hamiltonian
ops = FermionOperator((), 1.) + FermionOperator('1^ 1', 3.) + FermionOperator('1^ 2', 3. + 4.j) + FermionOperator('2^ 1', 3. - 4.j) + FermionOperator('3^ 4', 2. + 5.j) + FermionOperator('4^ 3', 2. - 5.j)
hamOF = get_quadratic_hamiltonian(ops, -0.03)
hamFQE = fqe.get_hamiltonian_from_openfermion(hamOF)
hamFQE = fqe.get_quadratic_hamiltonian(ops, -0.03)

Symmetry is passed in by lists containing permutation operations, a phase and whether or not complex conjugation is used.  Below are examples of a Hermetian matrix and a real anti-symmetric matrix.

In [18]:
e = 1.0
t = numpy.random.rand(4, 4) + numpy.random.rand(4, 4)*1.j
h = t + numpy.conjugate(t.T)
symmh = [
    [[1, 2], 1.0, False],
    [[2, 1], 1.0, True]
]
t = numpy.random.rand(4, 4, 4, 4)
g = t - t.T
symmg = [
    [[1, 2, 3, 4], 1.0, True],
    [[4, 3, 2, 1], -1.0, True]
]
hamFQE = fqe.get_two_body_hamiltonian(-0.3, h, g, 0.0, symmh, symmg)

The FQE will also return PolynomialTensor objects for parsing by OpenFermion objects.

In [19]:
hamOF = fqe.hamiltonian_to_openfermion(hamFQE)

### Saving and Loading Wavefunctions

Saving or loading a wavefunction is accomplished by passing a file name to the API and optionally a path to that filename.  If no path is given, the current working directory will be used.  If a path is used, the file will be searched for as

path + '/' + filename

In [20]:
wfnFqe = fqe.Wavefunction([[2, 2, 2], [2, 0, 2], [2, -2, 2]])
wfnFqe.set_wfn(strategy='random')
wfnFqe.print_wfn()


Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.06611546128988266+0.9978119730949402j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.06242252141237259-0.26834747195243835j)
 a'01'b'10' : (-0.02466556988656521-0.525239109992981j)
 a'10'b'01' : (-0.04141179472208023+0.5503657460212708j)
 a'10'b'10' : (-0.5428369641304016+0.21981625258922577j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (0.7478063106536865+0.6639169454574585j)


In [21]:
wfnFqe.save('random_wavefunction')
# wfnFqe.save('random_wavefunction', path='/home/google/wavefunctions')

Once it is saved you can return to the wavefunction by creating new object and then 'read' the data into it with the same syntax.

In [22]:
progress = fqe.Wavefunction()
progress.read('random_wavefunction')
# progress.read('random_wavefunction', path='/home/google/wavefunctions')
progress.print_wfn()


Configuration nelectrons: 2 m_s: -2
Vector : 0
 a'00'b'11' : (-0.06611546128988266+0.9978119730949402j)

Configuration nelectrons: 2 m_s: 0
Vector : 0
 a'01'b'01' : (0.06242252141237259-0.26834747195243835j)
 a'01'b'10' : (-0.02466556988656521-0.525239109992981j)
 a'10'b'01' : (-0.04141179472208023+0.5503657460212708j)
 a'10'b'10' : (-0.5428369641304016+0.21981625258922577j)

Configuration nelectrons: 2 m_s: 2
Vector : 0
 a'11'b'00' : (0.7478063106536865+0.6639169454574585j)
